In [1]:
import pandas as pd
import numpy as np
import os
import glob
import json
import re
from collections import defaultdict

import torchtext
from torchtext.legacy.data import Example

from wikipedia2vec import Wikipedia2Vec

In [2]:
# Data with Entities

ROOT_FOLDER = "/raid6/home/ramraj/2021/ir/entity-reranking/Entity-Linking/"

# BENCHMARK_TRAIN_FOLD_FOLDER = os.path.join(ROOT_FOLDER, "benchmark-train-relevance-v2.0")
# BENCHMARK_TEST_FILE = os.path.join(ROOT_FOLDER, "test-data", "ramraj-test-data-top100-BM25.json")
BENCHMARK_TRAIN_FOLD_FOLDER = os.path.join(ROOT_FOLDER, "Train-with-entities")
BENCHMARK_TEST_FILE = os.path.join(ROOT_FOLDER, "Test-with-entities/ramraj-test-data-top100-BM25-opt.json")

# BENCHMARK_TRAIN_FOLD_FOLDER = os.path.join(ROOT_FOLDER, "Train-with-entities-sepEntAndWords")
# BENCHMARK_TEST_FILE = os.path.join(ROOT_FOLDER, "Test-with-entities/ramraj-test-data-top100-BM25-opt-entWordSEP.json")

# laod train

In [3]:
train_files = sorted(glob.glob(os.path.join(BENCHMARK_TRAIN_FOLD_FOLDER, "fold-*.json")))
print( len(train_files) )

train_data = []
for train_file in train_files:
    tmp_data = json.load(open(train_file, 'r'))
    train_data.extend(tmp_data)
    
len(train_data)

5


1937

# load test

In [4]:
test_data = json.load(open(BENCHMARK_TEST_FILE, 'r'))
len(test_data)

2254

# utils

In [6]:
# def tokenise(text):
#     # Replace annoying unicode with a space
#     text = re.sub(r'[^\x00-\x7F]+', ' ', text)
#     # The following replacements are suggested in the paper
#     # BidAF (Seo et al., 2016)
#     text = text.replace("''", '" ')
#     text = text.replace("``", '" ')

#     # Space out punctuation
#     space_list = "!\"#$%&()*+,-./:;<=>?@[\\]^_`{|}~"
#     space_list = "!\"#$%&()*+,./:;<=>?@[\\]^_`{|}~"
#     text = text.translate(str.maketrans({key: " {0} ".format(key) for key in space_list}))

#     # space out singlequotes a bit better (and like stanford)
#     text = text.replace("'", " '")
    
#     # use any APIs
#     text = text.replace('\t', '').replace('\n', '').lower().strip()
#     return text

In [5]:
def alpha_numeric_filtering(text):
    regex = re.compile('[^a-zA-Z]')
    #First parameter is the replacement, second parameter is your input string
    return regex.sub(' ', text)     

# load wikie

In [18]:
import torchtext.vocab as vocab

MODEL_FILE = "../entity-embedding-modelling/data/enwiki_20180420_300d.txt"
# https://github.com/pytorch/text/issues/722

custom_embeddings = vocab.Vectors(name = MODEL_FILE, cache="/raid6/home/ramraj/2021/ir/entity-reranking/entity-embedding-modelling/data/cache-enwiki")

In [9]:
# PKL_MODEL_FILE = "../entity-embedding-modelling/data/enwiki_20180420_100d.pkl"
# wiki2vec = Wikipedia2Vec.load(PKL_MODEL_FILE)

# vocab

In [6]:
PAD_TOKEN = '<PAD>'
SOS_TOKEN = '<SOS>'
UNK_TOKEN = '<UNK>'
EOS_TOKEN = '<EOS>'

MAX_Q_LENGTH = 20
MAX_D_LEN = 200

Q_TEXT = torchtext.legacy.data.Field(sequential=True, 
                                tokenize=lambda x: x.split('<SEP>'),
                                eos_token=EOS_TOKEN,
                                pad_token=PAD_TOKEN,
                                include_lengths=True,
                                batch_first=True,
                                fix_length=MAX_Q_LENGTH,
                                lower=True)

DOC_TEXT = torchtext.legacy.data.Field(sequential=True, 
                                tokenize=lambda x: x.split('<SEP>'),
                                eos_token=EOS_TOKEN,
                                pad_token=PAD_TOKEN,
                                include_lengths=True,
                                batch_first=True,
                                fix_length=MAX_D_LEN,
                                lower=True)

In [21]:
example_texts = []

FIELDS = [('doc_text', DOC_TEXT), ('q_text', Q_TEXT)]
# https://github.com/pytorch/text/issues/722


for train_data_sample in train_data:
    # q_text = train_data_sample['qString']
    q_text = '<SEP>'.join( [ e['entity_title'] for e in train_data_sample['qEntities'] ] )
    
    for rel_docs in train_data_sample['RelevantDocuments']:
        # doc_text = rel_docs['docText']
        # dEntities = [ e['entity_title'] for e in rel_docs['dEntities'] ]
        doc_text = '<SEP>'.join( [ e['entity_title'] for e in rel_docs['dEntities'] ] )

        
        example_texts.append( Example.fromlist([doc_text, q_text] , FIELDS ) )
        
for test_data_sample in test_data:
    # q_text = test_data_sample['qString']
    q_text = '<SEP>'.join( [ e['entity_title'] for e in test_data_sample['qEntities'] ] )
    
    for rel_docs in test_data_sample['RelevantDocuments']:
        # doc_text = rel_docs['docText']
        doc_text = '<SEP>'.join( [ e['entity_title'] for e in rel_docs['dEntities'] ] )
        
        example_texts.append( Example.fromlist([doc_text, q_text] , FIELDS ) )

In [22]:
torchtext_dataset = torchtext.legacy.data.Dataset(example_texts, fields=FIELDS)

Q_TEXT.build_vocab(torchtext_dataset, vectors=custom_embeddings, vectors_cache="/raid6/home/ramraj/2021/ir/entity-reranking/entity-embedding-modelling/data/cache-enwiki")
DOC_TEXT.build_vocab(torchtext_dataset, vectors=custom_embeddings, vectors_cache="/raid6/home/ramraj/2021/ir/entity-reranking/entity-embedding-modelling/data/cache-enwiki")

print("======= Query ========")
print(len(Q_TEXT.vocab) )
print(Q_TEXT.vocab.vectors.size() )
print("======= Doc ========")
print( len(DOC_TEXT.vocab))
print( DOC_TEXT.vocab.vectors.size())

======= Query ========
2943
torch.Size([2943, 300])
======= Doc ========
171341
torch.Size([171341, 300])


In [23]:
print( DOC_TEXT.vocab.stoi['university of southampton'], DOC_TEXT.vocab.itos[14] )
print( DOC_TEXT.vocab.stoi['fahrenheit'], DOC_TEXT.vocab.itos[3] )

11557 disease
9974 human trafficking


# dataset

In [25]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch

from pytorch_transformers import WarmupLinearSchedule, AdamW
from transformers import get_linear_schedule_with_warmup

def map_to_torch_float(encoding):
    encoding = torch.FloatTensor(encoding)
    encoding.requires_grad_(False)
    return encoding

def map_to_torch(encoding):
    encoding = torch.LongTensor(encoding)
    encoding.requires_grad_(False)
    return encoding


class TestTRECDataset(Dataset):
    def __init__(self, raw_data, q_field, d_field, query_max_len=20, max_len=512, is_train=True):
        self.is_train = is_train
        self.max_len = max_len
        self.query_max_len = query_max_len
        
        self.q_field = q_field
        self.d_field = d_field

        self.qLookup = defaultdict()
        self.dLookup = defaultdict()
        self.data_df = self.load_data(raw_data)
        self.len = len(self.data_df)
        
    def load_data(self, data):
        return_data = {'qID': [], 'qText': [], 'dID': [], 'dText': [], 'label': []}
        for data_sample in data:
            qID = data_sample['qID']
            # qText = data_sample['qString']
            qText = '<SEP>'.join( [ e['entity_title'] for e in data_sample['qEntities'] ] )

            if qText == '': continue
            self.qLookup[qID] = qText

            for rel_docs in data_sample['RelevantDocuments']:
                dID = rel_docs['docID']
                # docText = rel_docs['docText']
                docText = '<SEP>'.join( [ e['entity_title'] for e in rel_docs['dEntities'] ] )
                
                if docText == '': continue
                self.dLookup[dID] = docText

                return_data['qID'].append(qID)
                return_data['dID'].append(dID)
                # return_data['qText'].append( qText )
                # return_data['dText'].append( docText )

        # df = pd.DataFrame(return_data, columns=["qID", "qText", "dID", "dText"])
        df = pd.DataFrame(return_data, columns=["qID", "dID"])
        return df
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        index = index % self.len
        inst = self.data_df.loc[index]
        docID = inst['dID']
        qID = inst['qID']
        docText = self.dLookup[docID] # self.dLookup[docID] # inst['dText']
        qText = self.qLookup[qID] # self.qLookup[qID] # inst['qText']
        
        # print(qID)
        # print(qText)
        # print()
        # print(docID)
        # print(docText)

        # =============== prepare text before feed ===============
        q_text_processed = self.q_field.numericalize( self.q_field.pad([ [l for l in self.q_field.preprocess(qText) if l.strip()] ]) )
        d_text_processed = self.d_field.numericalize( self.d_field.pad([ [l for l in self.d_field.preprocess(docText) if l.strip()] ]) )

        return q_text_processed, d_text_processed, qID, docID
    

test_trec_dataset = TestTRECDataset(test_data, Q_TEXT, DOC_TEXT)
print( test_trec_dataset.__len__() )

q, d, qID, dID = test_trec_dataset.__getitem__(1)

print(q[0].shape)
print(d[0].shape)
# print(q[0][0].shape)
# print(out[1][0].shape)
# print(out[2][0].shape)
# print()
# print(out[0][0].shape)

223331
torch.Size([1, 20])
torch.Size([1, 200])


In [26]:
q_str = []
for qq in q[0][0]:
    q_str.append(Q_TEXT.vocab.itos[qq])
print(' '.join( q_str) )
print()

q_str = []
for qq in d[0][0]:
    q_str.append(DOC_TEXT.vocab.itos[qq])
print(' '.join( q_str) )
print()

aftertaste <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

aftertaste food drink perception food drink mouth food drink tin can aftertaste perception food drink neuroscience evolution aftertaste signal transduction taste receptor adrenergic receptor mouth brain food processing insular cortex aftertaste perception <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 

# Model

In [28]:
import typing
import torch.nn as nn
import torch.nn.functional as F


class SimScorer(nn.Module):
    def __init__(self, q_text_field, d_text_field, with_multi_layer_perceptron=True, mlp_num_layers=2, mlp_num_units=300, mlp_num_fan_out=1, padding_idx=1, embedding_freeze=True):
        super(SimScorer, self).__init__()
        self.with_multi_layer_perceptron = with_multi_layer_perceptron

        self.mlp_num_layers = mlp_num_layers
        self.mlp_num_units = mlp_num_units
        self.mlp_num_fan_out = mlp_num_fan_out
        self.q_text_field = q_text_field
        self.d_text_field = d_text_field
        self.padding_idx = padding_idx
        self.embedding_freeze = embedding_freeze
        self.build()
            
    def build(self):
        """
        Build model structure.
        """
        self.q_embedding = nn.Embedding.from_pretrained(self.q_text_field.vocab.vectors, padding_idx=self.padding_idx, freeze=self.embedding_freeze)
        self.d_embedding = nn.Embedding.from_pretrained(self.d_text_field.vocab.vectors, padding_idx=self.padding_idx, freeze=self.embedding_freeze)

    def forward(self, inputs):
        input_left, input_right = inputs['text_left'], inputs['text_right']
        
        input_left_emb = self.q_embedding(input_left)
        input_right_emb = self.d_embedding(input_right)
        
        # print("input_left_emb : ", input_left_emb.shape)
        # print("input_right_emb : ", input_right_emb.shape)
        
        input_left_emb_avg = torch.mean(input_left_emb, dim=1)
        input_right_emb_avg = torch.mean(input_right_emb, dim=1)
        
        # print("input_left_emb_avg : ", input_left_emb_avg.shape)
        # print("input_right_emb_avg : ", input_right_emb_avg.shape)
    
        # Dot product with cosine similarity.
        x = F.cosine_similarity(input_left_emb_avg, input_right_emb_avg)
        # x = x.unsqueeze(dim=1)
        # print("x : ", x.shape)

        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

net = SimScorer(Q_TEXT, DOC_TEXT)
net.to(device)
print('Models built and ready to go!')

# tmp_q_text = torch.randint(low=1, high=30, size=(16, 20))
# tmp_d_text = torch.randint(low=1, high=30, size=(16, 200))
# print(tmp_q_text.shape)
# print(tmp_d_text.shape)

# input_text = {'text_left': tmp_q_text, 'text_right': tmp_d_text}

# output = net.forward(input_text)
# print(output)
# output.shape

cuda
Models built and ready to go!


In [29]:
# import config
import random
from bunch import Bunch
import time
import datetime

CONFIG_FILE = "config.json"

def format_time(elapsed_time):
    """
    Takes a time in seconds and returns a string hh:mm:ss
    """
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed_time)))
    return str(datetime.timedelta(seconds=elapsed_rounded)) # Format as hh:mm:ss

def get_config_from_json(json_file):
    """
        Get the config from a json file
        :param json_file:
        :return: config(namespace) or config(dictionary)
        """
    # parse the configurations from the config json file provided
    with open(json_file, 'r') as config_file:
        config_dict = json.load(config_file)

    # convert the dictionary to a namespace using bunch lib
    config = Bunch(config_dict)

    return config, config_dict

import torch
config, _ = get_config_from_json(CONFIG_FILE)
seed_val = config.cmd_args['seed']
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


config.data['results_dir'] = 'exp2_averageOf_cosineSim_entityOnly'



os.makedirs(config.data['results_dir'], exist_ok=True)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)


test_dataloader = DataLoader(dataset=test_trec_dataset,
                              batch_size=config.training["test_batch_size"],
                              pin_memory=device,
                              num_workers=config.training['num_workers'],
                              shuffle=True)
n_test_batches = len(test_dataloader)
print("Number of test batches : ", n_test_batches)
net.eval()

Number of test batches :  873


SimScorer(
  (q_embedding): Embedding(2943, 300, padding_idx=1)
  (d_embedding): Embedding(171341, 300, padding_idx=1)
)

# test

In [30]:
qID_list = []
paraID_list = []
pScore_list = []
t1 = time.time()
for batch_idx, test_batch_data in enumerate(test_dataloader):
    # Converting these to cuda tensors
    q_text, d_text, qID, passageID = test_batch_data
#     print(type(q_text))
#     x = q_text[0]
#     y = q_text[1]
#     print(x.shape)
#     print(y.shape)
#     break
    q_text, d_text = torch.squeeze(q_text[0].to(device)), torch.squeeze(d_text[0]).to(device)

    with torch.no_grad():
        net_output = net({'text_left': q_text, 'text_right': d_text})
        net_output = net_output.detach().cpu().numpy()

        for i in range(len(qID)):
            qID_list.append(qID[i])
            paraID_list.append(passageID[i])
            pScore_list.append(net_output[i])
    elapsed = format_time(time.time() - t1)
    if batch_idx % 50 == 0:
        print('  Batch {:>5,}  of  {:>5,}  :  processed    Elapsed: {:}.'.format(batch_idx,
                                                                                 n_test_batches,
                                                                                 elapsed))

pScore_list = [float(e) for e in pScore_list]
predicted_df = pd.DataFrame({"qID": qID_list,
                             "pID": paraID_list,
                             "pScore": pScore_list}, columns=["qID", "pID", "pScore"])
if not config.cmd_args['mode'] == "experiment":
    predicted_df.to_csv(os.path.join(config.data['results_dir'], "predictions.csv"))
print()

  Batch     0  of    873  :  processed    Elapsed: 0:00:00.
  Batch    50  of    873  :  processed    Elapsed: 0:00:07.
  Batch   100  of    873  :  processed    Elapsed: 0:00:12.
  Batch   150  of    873  :  processed    Elapsed: 0:00:18.
  Batch   200  of    873  :  processed    Elapsed: 0:00:24.
  Batch   250  of    873  :  processed    Elapsed: 0:00:29.
  Batch   300  of    873  :  processed    Elapsed: 0:00:35.
  Batch   350  of    873  :  processed    Elapsed: 0:00:41.
  Batch   400  of    873  :  processed    Elapsed: 0:00:46.
  Batch   450  of    873  :  processed    Elapsed: 0:00:52.
  Batch   500  of    873  :  processed    Elapsed: 0:00:57.
  Batch   550  of    873  :  processed    Elapsed: 0:01:03.
  Batch   600  of    873  :  processed    Elapsed: 0:01:12.
  Batch   650  of    873  :  processed    Elapsed: 0:01:17.
  Batch   700  of    873  :  processed    Elapsed: 0:01:22.
  Batch   750  of    873  :  processed    Elapsed: 0:01:29.
  Batch   800  of    873  :  processed  

In [31]:
# ================================================
#               Reverse Sorting Relevance
# ================================================
predicted_df = predicted_df[['qID', 'pID', 'pScore']]
grouped_pred_df = predicted_df.groupby(["qID"])
num_queries = len(grouped_pred_df)
missing_q_sets = 0
save_ranked_file = os.path.join(config.data['results_dir'], "ranked.test.relevance.txt")
with open(save_ranked_file, 'w') as write_file:
    q_cnt = 1
    for name, row_group in grouped_pred_df:
        rank_cnt = 1

        # ======= SORTING =======
        sorted_row_group = row_group.sort_values(by='pScore', ascending=False, inplace=False)
        # =======================

        if len(sorted_row_group) != 100:
            # print(">>>>>>>>>>> Missing query %s with shape %s" % (name, sorted_row_group.shape))
            # print(">>>>>>>>>>> Missing query with size %s" % sorted_row_group.shape[0])
            missing_q_sets += 1

        for i, row in sorted_row_group.iterrows():
            write_file.write("%s\tQ0\t%s\t%s\t%s\trchan31\n" % \
                             (row["qID"], row["pID"], rank_cnt, row["pScore"]))
            rank_cnt += 1

        print("Finished composing for query number : %s / %s" % (q_cnt, num_queries))
        q_cnt += 1
print()
print("Missing query-doc pairs : ", missing_q_sets)
print("Done train, val, and test !!!")

Finished composing for query number : 1 / 2236
Finished composing for query number : 2 / 2236
Finished composing for query number : 3 / 2236
Finished composing for query number : 4 / 2236
Finished composing for query number : 5 / 2236
Finished composing for query number : 6 / 2236
Finished composing for query number : 7 / 2236
Finished composing for query number : 8 / 2236
Finished composing for query number : 9 / 2236
Finished composing for query number : 10 / 2236
Finished composing for query number : 11 / 2236
Finished composing for query number : 12 / 2236
Finished composing for query number : 13 / 2236
Finished composing for query number : 14 / 2236
Finished composing for query number : 15 / 2236
Finished composing for query number : 16 / 2236
Finished composing for query number : 17 / 2236
Finished composing for query number : 18 / 2236
Finished composing for query number : 19 / 2236
Finished composing for query number : 20 / 2236
Finished composing for query number : 21 / 2236
F

Finished composing for query number : 181 / 2236
Finished composing for query number : 182 / 2236
Finished composing for query number : 183 / 2236
Finished composing for query number : 184 / 2236
Finished composing for query number : 185 / 2236
Finished composing for query number : 186 / 2236
Finished composing for query number : 187 / 2236
Finished composing for query number : 188 / 2236
Finished composing for query number : 189 / 2236
Finished composing for query number : 190 / 2236
Finished composing for query number : 191 / 2236
Finished composing for query number : 192 / 2236
Finished composing for query number : 193 / 2236
Finished composing for query number : 194 / 2236
Finished composing for query number : 195 / 2236
Finished composing for query number : 196 / 2236
Finished composing for query number : 197 / 2236
Finished composing for query number : 198 / 2236
Finished composing for query number : 199 / 2236
Finished composing for query number : 200 / 2236
Finished composing f

Finished composing for query number : 355 / 2236
Finished composing for query number : 356 / 2236
Finished composing for query number : 357 / 2236
Finished composing for query number : 358 / 2236
Finished composing for query number : 359 / 2236
Finished composing for query number : 360 / 2236
Finished composing for query number : 361 / 2236
Finished composing for query number : 362 / 2236
Finished composing for query number : 363 / 2236
Finished composing for query number : 364 / 2236
Finished composing for query number : 365 / 2236
Finished composing for query number : 366 / 2236
Finished composing for query number : 367 / 2236
Finished composing for query number : 368 / 2236
Finished composing for query number : 369 / 2236
Finished composing for query number : 370 / 2236
Finished composing for query number : 371 / 2236
Finished composing for query number : 372 / 2236
Finished composing for query number : 373 / 2236
Finished composing for query number : 374 / 2236
Finished composing f

Finished composing for query number : 523 / 2236
Finished composing for query number : 524 / 2236
Finished composing for query number : 525 / 2236
Finished composing for query number : 526 / 2236
Finished composing for query number : 527 / 2236
Finished composing for query number : 528 / 2236
Finished composing for query number : 529 / 2236
Finished composing for query number : 530 / 2236
Finished composing for query number : 531 / 2236
Finished composing for query number : 532 / 2236
Finished composing for query number : 533 / 2236
Finished composing for query number : 534 / 2236
Finished composing for query number : 535 / 2236
Finished composing for query number : 536 / 2236
Finished composing for query number : 537 / 2236
Finished composing for query number : 538 / 2236
Finished composing for query number : 539 / 2236
Finished composing for query number : 540 / 2236
Finished composing for query number : 541 / 2236
Finished composing for query number : 542 / 2236
Finished composing f

Finished composing for query number : 703 / 2236
Finished composing for query number : 704 / 2236
Finished composing for query number : 705 / 2236
Finished composing for query number : 706 / 2236
Finished composing for query number : 707 / 2236
Finished composing for query number : 708 / 2236
Finished composing for query number : 709 / 2236
Finished composing for query number : 710 / 2236
Finished composing for query number : 711 / 2236
Finished composing for query number : 712 / 2236
Finished composing for query number : 713 / 2236
Finished composing for query number : 714 / 2236
Finished composing for query number : 715 / 2236
Finished composing for query number : 716 / 2236
Finished composing for query number : 717 / 2236
Finished composing for query number : 718 / 2236
Finished composing for query number : 719 / 2236
Finished composing for query number : 720 / 2236
Finished composing for query number : 721 / 2236
Finished composing for query number : 722 / 2236
Finished composing f

...

Finished composing for query number : 1413 / 2236
Finished composing for query number : 1414 / 2236
Finished composing for query number : 1415 / 2236
Finished composing for query number : 1416 / 2236
Finished composing for query number : 1417 / 2236
Finished composing for query number : 1418 / 2236
Finished composing for query number : 1419 / 2236
Finished composing for query number : 1420 / 2236
Finished composing for query number : 1421 / 2236
Finished composing for query number : 1422 / 2236
Finished composing for query number : 1423 / 2236
Finished composing for query number : 1424 / 2236
Finished composing for query number : 1425 / 2236
Finished composing for query number : 1426 / 2236
Finished composing for query number : 1427 / 2236
Finished composing for query number : 1428 / 2236
Finished composing for query number : 1429 / 2236
Finished composing for query number : 1430 / 2236
Finished composing for query number : 1431 / 2236
Finished composing for query number : 1432 / 2236


Finished composing for query number : 1579 / 2236
Finished composing for query number : 1580 / 2236
Finished composing for query number : 1581 / 2236
Finished composing for query number : 1582 / 2236
Finished composing for query number : 1583 / 2236
Finished composing for query number : 1584 / 2236
Finished composing for query number : 1585 / 2236
Finished composing for query number : 1586 / 2236
Finished composing for query number : 1587 / 2236
Finished composing for query number : 1588 / 2236
Finished composing for query number : 1589 / 2236
Finished composing for query number : 1590 / 2236
Finished composing for query number : 1591 / 2236
Finished composing for query number : 1592 / 2236
Finished composing for query number : 1593 / 2236
Finished composing for query number : 1594 / 2236
Finished composing for query number : 1595 / 2236
Finished composing for query number : 1596 / 2236
Finished composing for query number : 1597 / 2236
Finished composing for query number : 1598 / 2236


Finished composing for query number : 1758 / 2236
Finished composing for query number : 1759 / 2236
Finished composing for query number : 1760 / 2236
Finished composing for query number : 1761 / 2236
Finished composing for query number : 1762 / 2236
Finished composing for query number : 1763 / 2236
Finished composing for query number : 1764 / 2236
Finished composing for query number : 1765 / 2236
Finished composing for query number : 1766 / 2236
Finished composing for query number : 1767 / 2236
Finished composing for query number : 1768 / 2236
Finished composing for query number : 1769 / 2236
Finished composing for query number : 1770 / 2236
Finished composing for query number : 1771 / 2236
Finished composing for query number : 1772 / 2236
Finished composing for query number : 1773 / 2236
Finished composing for query number : 1774 / 2236
Finished composing for query number : 1775 / 2236
Finished composing for query number : 1776 / 2236
Finished composing for query number : 1777 / 2236


Finished composing for query number : 1922 / 2236
Finished composing for query number : 1923 / 2236
Finished composing for query number : 1924 / 2236
Finished composing for query number : 1925 / 2236
Finished composing for query number : 1926 / 2236
Finished composing for query number : 1927 / 2236
Finished composing for query number : 1928 / 2236
Finished composing for query number : 1929 / 2236
Finished composing for query number : 1930 / 2236
Finished composing for query number : 1931 / 2236
Finished composing for query number : 1932 / 2236
Finished composing for query number : 1933 / 2236
Finished composing for query number : 1934 / 2236
Finished composing for query number : 1935 / 2236
Finished composing for query number : 1936 / 2236
Finished composing for query number : 1937 / 2236
Finished composing for query number : 1938 / 2236
Finished composing for query number : 1939 / 2236
Finished composing for query number : 1940 / 2236
Finished composing for query number : 1941 / 2236


Finished composing for query number : 2100 / 2236
Finished composing for query number : 2101 / 2236
Finished composing for query number : 2102 / 2236
Finished composing for query number : 2103 / 2236
Finished composing for query number : 2104 / 2236
Finished composing for query number : 2105 / 2236
Finished composing for query number : 2106 / 2236
Finished composing for query number : 2107 / 2236
Finished composing for query number : 2108 / 2236
Finished composing for query number : 2109 / 2236
Finished composing for query number : 2110 / 2236
Finished composing for query number : 2111 / 2236
Finished composing for query number : 2112 / 2236
Finished composing for query number : 2113 / 2236
Finished composing for query number : 2114 / 2236
Finished composing for query number : 2115 / 2236
Finished composing for query number : 2116 / 2236
Finished composing for query number : 2117 / 2236
Finished composing for query number : 2118 / 2236
Finished composing for query number : 2119 / 2236


In [33]:
! ls exp2_averageOf_cosineSim_entityOnly

ranked.test.relevance.txt


In [34]:
! ../Eval/trec_eval-master/trec_eval ../Eval/qrelsY1-test.V2.0/automatic-test.pages.cbor-hierarchical.qrels exp2_averageOf_cosineSim_entityOnly/ranked.test.relevance.txt  -m all_trec



runid                 	all	rchan31
num_q                 	all	2236
num_ret               	all	223331
num_rel               	all	6123
num_rel_ret           	all	2349
map                   	all	0.0594
gm_map                	all	0.0023
Rprec                 	all	0.0348
bpref                 	all	0.4689
recip_rank            	all	0.0944
iprec_at_recall_0.00  	all	0.0970
iprec_at_recall_0.10  	all	0.0967
iprec_at_recall_0.20  	all	0.0905
iprec_at_recall_0.30  	all	0.0768
iprec_at_recall_0.40  	all	0.0631
iprec_at_recall_0.50  	all	0.0616
iprec_at_recall_0.60  	all	0.0441
iprec_at_recall_0.70  	all	0.0427
iprec_at_recall_0.80  	all	0.0386
iprec_at_recall_0.90  	all	0.0382
iprec_at_recall_1.00  	all	0.0382
P_5                   	all	0.0268
P_10                  	all	0.0215
P_15                  	all	0.0193
P_20                  	all	0.0174
P_30                  	all	0.0157
P_100                 	all	0.0105
P_200                 	all	0.0053
P_500                 	all	0.0021
P_1000             

In [35]:
! ../Eval/trec_eval-master/trec_eval ../Eval/qrelsY1-test.V2.0/automatic-test.pages.cbor-hierarchical.qrels exp2_averageOf_cosineSim_entityOnly/ranked.test.relevance.txt  -m all_trec > exp2_averageOf_cosineSim_entityOnly_trec_results.txt


